In [21]:
from oyv.imports import *
import os
import gsw

In [22]:
os.chdir('/home/oyvindl/work/code/git_public/ice_ocean_plume/')
%run ice_ocean_plume.py
os.chdir('/home/oyvindl/work/projects/barents_glaciers/glacierocean-austfonna/'
         'plume_model/data/make_profiles/idealized_constantN')
dummy_ = np.arange(200)
ioplume = plume(10, 100, dummy_, dummy_, dummy_, 2)
ioplume.set_params()

In [23]:
pwd

'/home/oyvindl/work/projects/barents_glaciers/glacierocean-austfonna/plume_model/data/make_profiles/idealized_constantN'

In [24]:
def get_tempsal(DEP, TTOP, TBTM, SBTM, N2, dz =1):
    dep = np.arange(0, DEP+dz/2, dz)
    n = len(dep)
    temp = TTOP + np.arange(n)/n*(TBTM-TTOP)
    
    dtemp_dz = -np.diff(temp)/dz
    dsal_dz = -(ioplume.BT*dtemp_dz - N2/ioplume.g)/ioplume.BS
    sal = np.concatenate([np.array([SBTM]), SBTM + np.cumsum(dsal_dz)])
    
    return temp, sal, dep

In [25]:
DEP = 100
TTOP = 0
TBTM = 2
SBTM = 34.7
N2 = 1e-4

In [26]:
dz = 1
dep = np.arange(0, DEP+dz/2, dz)
n = len(dep)
temp = TTOP + np.arange(n)/n*(TBTM-TTOP)

In [27]:
dtemp_dz = -np.diff(temp)/dz
dsal_dz = -(ioplume.BT*dtemp_dz - N2/ioplume.g)/ioplume.BS

In [28]:
sal = np.concatenate([np.array([SBTM]), SBTM + np.cumsum(dsal_dz)])

In [29]:
rho = gsw.rho?


In [30]:
lat = 80
pres = gsw.p_from_z(-dep, lat)
SA = gsw.SA_from_SP(sal, pres, 30, lat)
CT = gsw.CT_from_t(SA, temp, pres)
sig0 = gsw.sigma0(SA, CT)
rho = gsw.rho(SA, CT, pres)
n2, middep = gsw.Nsquared(SA, CT, pres, lat = lat)

In [31]:
n2 = ioplume.g/rho.mean()*np.diff(sig0)

In [32]:
ca()

In [33]:
fig, ax = plt.subplots(1, 4, sharey = True)
ax[0].plot(temp, dep)
ax[0].plot(CT, dep)

ax[1].plot(sal, dep)
ax[1].plot(SA, dep)

ax[2].plot(rho, dep)
ax[3].plot(n2, middep)
pl.vline(N2, ax = ax[3])
#ax[3].set_xlim(N2*0.5, N2*1.5)
ax[0].invert_yaxis()

In [34]:
def get_tempsal(DEP, TTOP, TBTM, SBTM, N2, dz =1, rho0 = 1027):
    dep = np.arange(0, DEP+dz/2, dz)
    n = len(dep)
    temp = TTOP + np.arange(n)/n*(TBTM-TTOP)
    
    dtemp_dz = -np.diff(temp)/dz
    dsal_dz = (ioplume.BT*dtemp_dz - N2/ioplume.g)/ioplume.BS
    sal = np.concatenate([np.array([SBTM]), SBTM + np.cumsum(dsal_dz)])[::-1]
    
    drho_dz = rho0*(ioplume.BS*dsal_dz - ioplume.BT*dtemp_dz)
    n2 = -ioplume.g/rho0*drho_dz
    return temp, sal, dep, n2

In [35]:

fig, ax = plt.subplots(1, 3, sharey = True)
N2 = 1e-5
temp, sal, dep, n2 = get_tempsal(DEP, 0, 2, SBTM, N2, dz =1)

pl.vline(N2, ax = ax[2])
ax[0].plot(temp, dep)
ax[1].plot(sal, dep)
ax[2].plot(n2, middep)
ax[0].invert_yaxis()

temp, sal, dep, n2 = get_tempsal(DEP, -2, 13, SBTM, N2, dz =1)
ax[0].plot(temp, dep, '--')
ax[1].plot(sal, dep, '--')
ax[2].plot(n2, middep, '--')



In [36]:
print('N2: %.2e'%N2)
print('n2: %.2e'%n2.mean())

N2: 1.00e-05
n2: 1.00e-05
